In [2]:
import pandas as pd
import re

with open("01_namespace_sample.log", "r", encoding="utf-8") as f:
    lines = f.readlines()

lines = [line.strip() for line in lines if line.strip()]

# заголовки
header_line = None
for line in lines:
    if line.startswith("NAME"):
        header_line = line
        break

if not header_line:
    print("Не удалось найти строку заголовка.")
else:
    # по 2+ пробелам
    headers = re.split(r'\s{2,}', header_line)
    
    # строки после инд.
    header_index = lines.index(header_line)
    data_rows = lines[header_index + 1:]
    
    data = []
    for row in data_rows:
        row_values = re.split(r'\s{2,}', row)
        # Если число знач. меньше, чем заголовков - пуст. знач.
        if len(row_values) < len(headers):
            row_values += [None] * (len(headers) - len(row_values))
        data.append(row_values)
    
    df_namespace = pd.DataFrame(data, columns=headers)
    print("Данные по namespace:")
    display(df_namespace)


Данные по namespace:


,NAME,READY,STATUS,RESTARTS,AGE,IP,NODE,NOMINATED NODE,READINESS GATES
0,config-manager-28961525-9ncbb,0/1,Completed,0,3d9h,100.100.18.111,ip-100-100-24-129.eu-west-1.compute.internal,<none>,<none>
1,config-manager-28965845-dkmjz,0/1,Completed,0,9h,100.100.31.170,ip-100-100-24-129.eu-west-1.compute.internal,<none>,<none>
2,config-manager-tecrp-g28p9,0/1,Completed,0,9m12s,100.100.47.38,ip-100-100-46-4.eu-west-1.compute.internal,<none>,<none>
3,mongodb-backup-28964280-9lfqk,0/1,Completed,0,35h,100.100.31.153,ip-100-100-24-129.eu-west-1.compute.internal,<none>,<none>
4,mongodb-backup-28965720-p75fx,0/1,Completed,0,11h,100.100.26.233,ip-100-100-24-129.eu-west-1.compute.internal,<none>,<none>
5,xenia-56b6dcb558-6hdql,2/2,Running,0,3d21h,100.100.22.196,ip-100-100-31-113.eu-west-1.compute.internal,<none>,<none>
6,xenia-56b6dcb558-sr4jr,2/2,Running,0,3d22h,100.100.38.210,ip-100-100-35-194.eu-west-1.compute.internal,<none>,<none>
7,xenia-stg-mongodb-0,3/3,Running,0,73d,100.100.32.141,ip-100-100-41-106.eu-west-1.compute.internal,<none>,<none>
8,xenia-stg-mongodb-1,3/3,Running,0,73d,100.100.62.149,ip-100-100-58-211.eu-west-1.compute.internal,<none>,<none>
9,xenia-stg-mongodb-2,3/3,Running,0,73d,100.100.22.110,ip-100-100-17-161.eu-west-1.compute.internal,<none>,<none>


In [3]:
import re

with open("02_describe_node.log", "r", encoding="utf-8") as f:
    node_lines = f.readlines()

# Allocatable: строки до первой пустой
allocatable_resources = {}
found_allocatable = False
for line in node_lines:
    if line.startswith("Allocatable:"):
        found_allocatable = True
        continue  # след. стр.
    if found_allocatable:
        # если встречается пустая строка, значит секция закончилась
        if line.strip() == "":
            break
        match = re.match(r'\s*(\S+):\s+(.*)', line)
        if match:
            key, value = match.groups()
            allocatable_resources[key] = value

print("Allocatable resources for the node:")
for key, value in allocatable_resources.items():
    print(f"{key}: {value}")


Allocatable resources for the node:
cpu: 3920m
ephemeral-storage: 27905944324
hugepages-1Gi: 0
hugepages-2Mi: 0
memory: 7005648Ki
pods: 58
Architecture: amd64
ProviderID: aws:///eu-west-1a/i-01a0e440b65a2910a
Events: <none>


In [4]:
import re
import pandas as pd

def parse_cpu(cpu_str):
    if not cpu_str:
        return 0.0
    cpu_str = cpu_str.strip()
    cpu_val = cpu_str.split()[0]
    if cpu_val.endswith("m"):
        try:
            return float(cpu_val[:-1])
        except ValueError:
            return 0.0
    else:
        try:
            return float(cpu_val) * 1000
        except ValueError:
            return 0.0

def parse_memory(mem_str):
    if not mem_str:
        return 0.0
    mem_str = mem_str.strip()
    mem_val = mem_str.split()[0]
    try:
        if mem_val.endswith("Ki"):
            return float(mem_val[:-2]) / 1024
        elif mem_val.endswith("Mi"):
            return float(mem_val[:-2])
        elif mem_val.endswith("Gi"):
            return float(mem_val[:-2]) * 1024
        else:
            return float(mem_val)
    except ValueError:
        return 0.0

def is_valid_pod_entry(row_dict):
    """
    True, если значение в 'Name' является корректным именем пода.
    Если значение отсутствует или содержит пробелы, скобки и т.д. – False.
    """
    name = row_dict.get("Name")
    if not name:
        return False
    if re.search(r'[\s\(\)]', name):
        return False
    return True

with open("02_describe_node.log", "r", encoding="utf-8") as f:
    node_lines = f.readlines()

# Non-terminated Pods
pods_section = []
in_pods_section = False
for line in node_lines:
    if "Non-terminated Pods:" in line:
        in_pods_section = True
        continue
    if in_pods_section:
        if line.strip() == "":
            break
        pods_section.append(line.rstrip())

print("Non-terminated Pods:")
for line in pods_section:
    print(line)

# Поиск строки заголовка
header_line = None
data_lines = []
for i, line in enumerate(pods_section):
    if line.strip().startswith("Namespace"):
        header_line = line.strip()
        data_lines = pods_section[i+2:]
        break

if not header_line:
    print("Не удалось найти заголовок таблицы в секции Non-terminated Pods.")
else:
    headers = re.split(r'\s{2,}', header_line)
    pod_data = []
    for row in data_lines:
        if re.match(r'^[-\s]+$', row):
            continue
        cols = re.split(r'\s{2,}', row.strip())
        if len(cols) < len(headers):
            cols += [None] * (len(headers) - len(cols))
        row_dict = dict(zip(headers, cols))
        if is_valid_pod_entry(row_dict):
            pod_data.append(cols)
    
    df_allocated = pd.DataFrame(pod_data, columns=headers)
    print("Сводная таблица Non-terminated Pods после фильтрации:")
    display(df_allocated)
    
    # Преобразование значений ресурсов в числовой формат
    df_allocated["CPU Requests_m"] = df_allocated["CPU Requests"].apply(parse_cpu)
    df_allocated["CPU Limits_m"] = df_allocated["CPU Limits"].apply(parse_cpu)
    df_allocated["Memory Requests_Mi"] = df_allocated["Memory Requests"].apply(parse_memory)
    df_allocated["Memory Limits_Mi"] = df_allocated["Memory Limits"].apply(parse_memory)
    
    total_cpu_requests = df_allocated["CPU Requests_m"].sum()
    total_cpu_limits = df_allocated["CPU Limits_m"].sum()
    total_memory_requests = df_allocated["Memory Requests_Mi"].sum()
    total_memory_limits = df_allocated["Memory Limits_Mi"].sum()
    
    print("Суммарно по всем подам на ноде:")
    print(f"CPU Requests: {total_cpu_requests} m")
    print(f"CPU Limits: {total_cpu_limits} m")
    print(f"Memory Requests: {total_memory_requests} MiB")
    print(f"Memory Limits: {total_memory_limits} MiB")
    
    # проверка какие поды занимают более 50% allocatable ресурсов.
    def parse_allocatable_cpu(cpu_str):
        if not cpu_str:
            return 0.0
        cpu_str = cpu_str.strip()
        if cpu_str.endswith("m"):
            return float(cpu_str[:-1])
        else:
            try:
                return float(cpu_str) * 1000
            except ValueError:
                return 0.0

    def parse_allocatable_memory(mem_str):
        if not mem_str:
            return 0.0
        mem_str = mem_str.strip()
        if mem_str.endswith("Ki"):
            return float(mem_str[:-2]) / 1024
        elif mem_str.endswith("Mi"):
            return float(mem_str[:-2])
        elif mem_str.endswith("Gi"):
            return float(mem_str[:-2]) * 1024
        else:
            return float(mem_str)
    
    alloc_cpu_m = parse_allocatable_cpu(allocatable_resources.get("cpu", "0"))
    alloc_mem_mi = parse_allocatable_memory(allocatable_resources.get("memory", "0"))
    
    pods_exceed_half = []
    for idx, row in df_allocated.iterrows():
        cpu_req = row["CPU Requests_m"]
        mem_req = row["Memory Requests_Mi"]
        cpu_ratio = (cpu_req / alloc_cpu_m * 100) if alloc_cpu_m > 0 else 0
        mem_ratio = (mem_req / alloc_mem_mi * 100) if alloc_mem_mi > 0 else 0
        if cpu_ratio > 50 or mem_ratio > 50:
            pods_exceed_half.append({
                "Namespace": row["Namespace"],
                "Name": row["Name"],
                "CPU_req_m": cpu_req,
                "Mem_req_Mi": mem_req,
                "CPU_ratio_%": cpu_ratio,
                "Mem_ratio_%": mem_ratio
            })
    if pods_exceed_half:
        print("\nПоды, которые требуют более 50% от allocatable ресурсов (CPU или Memory):")
        for pod in pods_exceed_half:
            print(pod)
    else:
        print("\nНет подов, которые занимают >50% от ресурсов ноды.")


Non-terminated Pods:
  Namespace                   Name                                         CPU Requests  CPU Limits   Memory Requests  Memory Limits  Age
  ---------                   ----                                         ------------  ----------   ---------------  -------------  ---
  calico-system               calico-node-brn8p                            0 (0%)        0 (0%)       0 (0%)           0 (0%)         3d22h
  calico-system               csi-node-driver-878fn                        0 (0%)        0 (0%)       0 (0%)           0 (0%)         3d22h
  falcon-system               falcon-sensor-k5xzq                          0 (0%)        0 (0%)       0 (0%)           0 (0%)         3d22h
  kube-system                 aws-node-cj5dj                               25m (0%)      0 (0%)       0 (0%)           0 (0%)         3d22h
  kube-system                 aws-node-termination-handler-g98fx           0 (0%)        0 (0%)       0 (0%)           0 (0%)         3d22h
  k

,Namespace,Name,CPU Requests,CPU Limits,Memory Requests,Memory Limits,Age
0,calico-system,calico-node-brn8p,0 (0%),0 (0%),0 (0%),0 (0%),3d22h
1,calico-system,csi-node-driver-878fn,0 (0%),0 (0%),0 (0%),0 (0%),3d22h
2,falcon-system,falcon-sensor-k5xzq,0 (0%),0 (0%),0 (0%),0 (0%),3d22h
3,kube-system,aws-node-cj5dj,25m (0%),0 (0%),0 (0%),0 (0%),3d22h
4,kube-system,aws-node-termination-handler-g98fx,0 (0%),0 (0%),0 (0%),0 (0%),3d22h
5,kube-system,ebs-csi-node-hsd8l,30m (0%),0 (0%),120Mi (1%),768Mi (11%),3d22h
6,kube-system,kube-proxy-ftgcs,100m (2%),0 (0%),0 (0%),0 (0%),3d22h
7,logging,fluent-bit-kszcv,20m (0%),2100m (53%),128Mi (1%),1Gi (14%),3d22h
8,monitoring,monitoring-prometheus-node-exporter-g9jch,0 (0%),0 (0%),0 (0%),0 (0%),3d22h
9,tigera-compliance,compliance-benchmarker-x8s8c,0 (0%),0 (0%),0 (0%),0 (0%),3d22h


Суммарно по всем подам на ноде:
CPU Requests: 2185.0 m
CPU Limits: 6100.0 m
Memory Requests: 2424.0 MiB
Memory Limits: 4864.0 MiB

Поды, которые требуют более 50% от allocatable ресурсов (CPU или Memory):
{'Namespace': 'xenia-stg', 'Name': 'xenia-56b6dcb558-6hdql', 'CPU_req_m': 2010.0, 'Mem_req_Mi': 2176.0, 'CPU_ratio_%': 51.275510204081634, 'Mem_ratio_%': 31.806108442787878}


In [5]:
import re

# 1. Memory request overcommit (Forbidden)
if total_memory_requests > alloc_mem_mi:
    print("Memory request overcommit DETECTED -> Forbidden!")
else:
    print("Memory request overcommit is NOT detected.")

# 2. Memory limits overcommit (Red flag)
if total_memory_limits > alloc_mem_mi:
    print("Memory limits overcommit DETECTED -> Red flag!")
else:
    print("Memory limits overcommit is NOT detected.")

# 3. CPU limits overcommit > 300%
# (total_cpu_limits / alloc_cpu_m) * 100
if alloc_cpu_m > 0: 
    cpu_limits_ratio_percent = (total_cpu_limits / alloc_cpu_m) * 100
    print(f"CPU limits ratio = {cpu_limits_ratio_percent:.2f}%")
    if cpu_limits_ratio_percent > 300:
        print("CPU limits overcommit > 300% -> Recommend to reorganize nodes!")
    else:
        print("CPU limits overcommit is NOT detected (>300%).")
else:
    print("Allocatable CPU = 0, пропускаем проверку CPU overcommit.")

# поды, которые требуют > 50% от ноды (по памяти или CPU).
if 'df_allocated' in globals():
    # проверяем для каждого pod
    pods_exceed_half = []
    for idx, row in df_allocated.iterrows():
        cpu_req = row["CPU Requests_m"]
        mem_req = row["Memory Requests_Mi"]
        # Проверка на 50% CPU
        cpu_ratio = (cpu_req / alloc_cpu_m * 100) if alloc_cpu_m > 0 else 0
        mem_ratio = (mem_req / alloc_mem_mi * 100) if alloc_mem_mi > 0 else 0
        if cpu_ratio > 50 or mem_ratio > 50:
            pods_exceed_half.append({
                "Namespace": row["Namespace"],
                "Name": row["Name"],
                "CPU_req_m": cpu_req,
                "Mem_req_Mi": mem_req,
                "CPU_ratio_%": cpu_ratio,
                "Mem_ratio_%": mem_ratio
            })
    if pods_exceed_half:
        print("\nПоды, которые требуют более 50% от allocatable ресурсов (CPU или Memory):")
        for pod in pods_exceed_half:
            print(pod)
    else:
        print("\nНет подов, которые занимают >50% от ресурсов ноды.")
else:
    print("DataFrame df_allocated не найден, пропускаем проверку подов на использование >50%.")


Memory request overcommit is NOT detected.
Memory limits overcommit is NOT detected.
CPU limits ratio = 155.61%
CPU limits overcommit is NOT detected (>300%).

Поды, которые требуют более 50% от allocatable ресурсов (CPU или Memory):
{'Namespace': 'xenia-stg', 'Name': 'xenia-56b6dcb558-6hdql', 'CPU_req_m': 2010.0, 'Mem_req_Mi': 2176.0, 'CPU_ratio_%': 51.275510204081634, 'Mem_ratio_%': 31.806108442787878}


In [5]:
# проверка, превышает ли запрос пода 100% allocatable ресурсов
def exceeds_100_percent(cpu_req, mem_req, alloc_cpu, alloc_mem):
    cpu_exceeds = (cpu_req / alloc_cpu * 100) > 100 if alloc_cpu > 0 else False
    mem_exceeds = (mem_req / alloc_mem * 100) > 100 if alloc_mem > 0 else False
    return cpu_exceeds or mem_exceeds

# проверка БД
def is_database_pod(pod_name):
    db_keywords = ["mongo", "redis", "postgres", "mysql", "couchbase"]
    if not pod_name:
        return False
    pod_name_lower = pod_name.lower()
    return any(keyword in pod_name_lower for keyword in db_keywords)

pods_exceed_100 = []
database_pods = []

# перебор строк df_allocated
for idx, row in df_allocated.iterrows():
    cpu_req = row["CPU Requests_m"]
    mem_req = row["Memory Requests_Mi"]
    pod_name = row["Name"]
    
    # соотношение в процентах
    cpu_ratio = (cpu_req / alloc_cpu_m * 100) if alloc_cpu_m > 0 else 0
    mem_ratio = (mem_req / alloc_mem_mi * 100) if alloc_mem_mi > 0 else 0
    
    # проверка превышает ли запрос 100% от доступных ресурсов
    if exceeds_100_percent(cpu_req, mem_req, alloc_cpu_m, alloc_mem_mi):
        pods_exceed_100.append({
            "Namespace": row["Namespace"],
            "Name": pod_name,
            "CPU_req_m": cpu_req,
            "Mem_req_Mi": mem_req,
            "CPU_ratio_%": cpu_ratio,
            "Mem_ratio_%": mem_ratio
        })
    
    # проверка БД
    if is_database_pod(pod_name):
        database_pods.append({
            "Namespace": row["Namespace"],
            "Name": pod_name,
            "CPU_req_m": cpu_req,
            "Mem_req_Mi": mem_req,
            "CPU_ratio_%": cpu_ratio,
            "Mem_ratio_%": mem_ratio
        })

print("\nПоды, у которых запрос превышает 100% allocatable ресурсов ноды (Validation Rule):")
if pods_exceed_100:
    for pod in pods_exceed_100:
        print(pod)
else:
    print("Нет подов, у которых запрос превышает 100% ресурсов ноды.")

print("\nОбнаружены поды, относящиеся к базам данных:")
if database_pods:
    for pod in database_pods:
        print(pod)
else:
    print("БД не обнаружены по именам подов.")



Поды, у которых запрос превышает 100% allocatable ресурсов ноды (Validation Rule):
Нет подов, у которых запрос превышает 100% ресурсов ноды.

Обнаружены поды, относящиеся к базам данных:
БД не обнаружены по именам подов.


In [2]:
import pandas as pd
import re
from IPython.display import display

# парс
def parse_cpu(cpu_str):
    if not cpu_str:
        return 0.0
    cpu_val = cpu_str.split()[0]
    try:
        if cpu_val.endswith("m"):
            return float(cpu_val[:-1])
        return float(cpu_val) * 1000
    except ValueError:
        return 0.0

def parse_memory(mem_str):
    if not mem_str:
        return 0.0
    mem_val = mem_str.split()[0]
    try:
        if mem_val.endswith("Ki"):
            return float(mem_val[:-2]) / 1024
        if mem_val.endswith("Mi"):
            return float(mem_val[:-2])
        if mem_val.endswith("Gi"):
            return float(mem_val[:-2]) * 1024
        return float(mem_val)
    except ValueError:
        return 0.0

def parse_allocatable_cpu(cpu_str):
    return parse_cpu(cpu_str)

def parse_allocatable_memory(mem_str):
    return parse_memory(mem_str)


with open("01_namespace_sample.log", "r", encoding="utf-8") as f:
    lines = [l.strip() for l in f if l.strip()]
header_line = next((l for l in lines if l.startswith("NAME")), None)
if header_line:
    headers = re.split(r'\s{2,}', header_line)
    idx = lines.index(header_line)
    rows = []
    for row in lines[idx+1:]:
        vals = re.split(r'\s{2,}', row)
        if len(vals) < len(headers):
            vals += [None] * (len(headers) - len(vals))
        rows.append(vals)
    df_namespace = pd.DataFrame(rows, columns=headers)
    print("Данные по namespace:")
    display(df_namespace)
else:
    print("Namespace data not found.")


with open("02_describe_node.log", "r", encoding="utf-8") as f:
    node_lines = f.readlines()

allocatable_resources = {}
found = False
for line in node_lines:
    if line.startswith("Allocatable:"):
        found = True
        continue
    if found:
        if not line.strip():
            break
        m = re.match(r'\s*(\S+):\s+(.+)', line)
        if m:
            allocatable_resources[m.group(1)] = m.group(2).strip()

print("Allocatable resources:")
print(allocatable_resources)


pods_section = []
start = False
for l in node_lines:
    if "Non-terminated Pods:" in l:
        start = True
        continue
    if start:
        if not l.strip():
            break
        pods_section.append(l.rstrip())

header = None
for i, l in enumerate(pods_section):
    if l.strip().startswith("Namespace"):
        header = re.split(r'\s{2,}', l.strip())
        data_lines = pods_section[i+2:]
        break

if header:
    data = []
    for row in data_lines:
        if re.match(r'^[-\s]+$', row):
            continue
        vals = re.split(r'\s{2,}', row.strip())
        if len(vals) < len(header):
            vals += [None] * (len(header) - len(vals))
        row_dict = dict(zip(header, vals))
        name = row_dict.get("Name")
        if name and not re.search(r'[\s\(\)]', name):
            data.append(vals)
    df_allocated = pd.DataFrame(data, columns=header)
    df_allocated["CPU Requests_m"]     = df_allocated["CPU Requests"].apply(parse_cpu)
    df_allocated["CPU Limits_m"]       = df_allocated["CPU Limits"].apply(parse_cpu)
    df_allocated["Memory Requests_Mi"] = df_allocated["Memory Requests"].apply(parse_memory)
    df_allocated["Memory Limits_Mi"]   = df_allocated["Memory Limits"].apply(parse_memory)
    print("Сводная таблица Non-terminated Pods:")
    display(df_allocated[["Namespace","Name","CPU Requests_m","CPU Limits_m","Memory Requests_Mi","Memory Limits_Mi"]])
else:
    print("Не удалось найти таблицу Non-terminated Pods.")

# affinity/anti-affinity
def can_place(pod, node):
    dbs = ("mongo","postgres","redis","mysql","couchbase")
    if any(any(db in p["name"].lower() for db in dbs) for p in node["pods"]):
        if any(db in pod["name"].lower() for db in dbs):
            return False
    return True

# weighted bin-pack
def optimize_node_binpack(df_allocated, alloc_res, alpha=1.0):
    pods = []
    for _, row in df_allocated.iterrows():
        pods.append({
            "name":    row["Name"],
            "cpu_req": int(round(row["CPU Requests_m"])),
            "mem_req": int(round(row["Memory Requests_Mi"])),
            "cpu_lim": int(round(row["CPU Limits_m"])),
            "mem_lim": int(round(row["Memory Limits_Mi"])),
        })

    alloc_cpu = parse_allocatable_cpu(alloc_res.get("cpu","0"))
    alloc_mem = parse_allocatable_memory(alloc_res.get("memory","0"))
    nodes = [{"name":"current-node","cpu":int(round(alloc_cpu)),
              "mem":int(round(alloc_mem)),"taints":{},"pods":[]}]

    def binpack_node_weighted(node, pool, alpha):
        C, M = node["cpu"], node["mem"]
        dp = [[(0.0, []) for _ in range(M+1)] for _ in range(C+1)]
        for idx, p in enumerate(pool):
            rc, rm = p["cpu_req"], p["mem_req"]
            lc, lm = p["cpu_lim"], p["mem_lim"]
            for c in range(C, rc-1, -1):
                for m in range(M, rm-1, -1):
                    val0, sel0 = dp[c-rc][m-rm]
                    sel1 = sel0 + [idx]
                    used = sum(pool[i]["cpu_req"]+pool[i]["mem_req"] for i in sel1)
                    tot_c = sum(pool[i]["cpu_lim"] for i in sel1)
                    tot_m = sum(pool[i]["mem_lim"] for i in sel1)
                    pen = alpha*max(0,tot_c-C) + alpha*max(0,tot_m-M)
                    v1 = used - pen
                    if v1 > dp[c][m][0]:
                        dp[c][m] = (v1, sel1)
        best_val, best_sel = 0.0, []
        for c in range(C+1):
            for m in range(M+1):
                v, sel = dp[c][m]
                if v > best_val:
                    best_val, best_sel = v, sel
        selected = [pool[i] for i in best_sel]
        rest     = [p for i,p in enumerate(pool) if i not in best_sel]
        node["pods"].extend(selected)
        return rest

    unscheduled = pods.copy()
    for node in nodes:
        print(f"=== Оптимизация для {node['name']} ===")
        # фильтруем по can_place и выполняем
        candidates     = [p for p in unscheduled if can_place(p, node)]
        not_candidates = [p for p in unscheduled if p not in candidates]
        rest_after_dp  = binpack_node_weighted(node, candidates, alpha)
        unscheduled    = not_candidates + rest_after_dp

    return nodes, unscheduled



nodes_res, unsch = optimize_node_binpack(df_allocated, allocatable_resources, alpha=2.0)

print("\n--- Итоговое распределение pod-ов ---")
for n in nodes_res:
    uc = sum(p["cpu_req"] for p in n["pods"])
    um = sum(p["mem_req"] for p in n["pods"])
    print(f"{n['name']}: CPU {uc}/{n['cpu']}  MEM {um}/{n['mem']}")
if unsch:
    print("Не влезли:", [p["name"] for p in unsch])
else:
    print("Все pod-ы распределены.")


Данные по namespace:


,NAME,READY,STATUS,RESTARTS,AGE,IP,NODE,NOMINATED NODE,READINESS GATES
0,config-manager-28961525-9ncbb,0/1,Completed,0,3d9h,100.100.18.111,ip-100-100-24-129.eu-west-1.compute.internal,<none>,<none>
1,config-manager-28965845-dkmjz,0/1,Completed,0,9h,100.100.31.170,ip-100-100-24-129.eu-west-1.compute.internal,<none>,<none>
2,config-manager-tecrp-g28p9,0/1,Completed,0,9m12s,100.100.47.38,ip-100-100-46-4.eu-west-1.compute.internal,<none>,<none>
3,mongodb-backup-28964280-9lfqk,0/1,Completed,0,35h,100.100.31.153,ip-100-100-24-129.eu-west-1.compute.internal,<none>,<none>
4,mongodb-backup-28965720-p75fx,0/1,Completed,0,11h,100.100.26.233,ip-100-100-24-129.eu-west-1.compute.internal,<none>,<none>
5,xenia-56b6dcb558-6hdql,2/2,Running,0,3d21h,100.100.22.196,ip-100-100-31-113.eu-west-1.compute.internal,<none>,<none>
6,xenia-56b6dcb558-sr4jr,2/2,Running,0,3d22h,100.100.38.210,ip-100-100-35-194.eu-west-1.compute.internal,<none>,<none>
7,xenia-stg-mongodb-0,3/3,Running,0,73d,100.100.32.141,ip-100-100-41-106.eu-west-1.compute.internal,<none>,<none>
8,xenia-stg-mongodb-1,3/3,Running,0,73d,100.100.62.149,ip-100-100-58-211.eu-west-1.compute.internal,<none>,<none>
9,xenia-stg-mongodb-2,3/3,Running,0,73d,100.100.22.110,ip-100-100-17-161.eu-west-1.compute.internal,<none>,<none>


Allocatable resources:
{'cpu': '3920m', 'ephemeral-storage': '27905944324', 'hugepages-1Gi': '0', 'hugepages-2Mi': '0', 'memory': '7005648Ki', 'pods': '58', 'Architecture': 'amd64', 'ProviderID': 'aws:///eu-west-1a/i-01a0e440b65a2910a', 'Events': '<none>'}
Сводная таблица Non-terminated Pods:


,Namespace,Name,CPU Requests_m,CPU Limits_m,Memory Requests_Mi,Memory Limits_Mi
0,calico-system,calico-node-brn8p,0.0,0.0,0.0,0.0
1,calico-system,csi-node-driver-878fn,0.0,0.0,0.0,0.0
2,falcon-system,falcon-sensor-k5xzq,0.0,0.0,0.0,0.0
3,kube-system,aws-node-cj5dj,25.0,0.0,0.0,0.0
4,kube-system,aws-node-termination-handler-g98fx,0.0,0.0,0.0,0.0
5,kube-system,ebs-csi-node-hsd8l,30.0,0.0,120.0,768.0
6,kube-system,kube-proxy-ftgcs,100.0,0.0,0.0,0.0
7,logging,fluent-bit-kszcv,20.0,2100.0,128.0,1024.0
8,monitoring,monitoring-prometheus-node-exporter-g9jch,0.0,0.0,0.0,0.0
9,tigera-compliance,compliance-benchmarker-x8s8c,0.0,0.0,0.0,0.0


=== Оптимизация для current-node ===

--- Итоговое распределение pod-ов ---
current-node: CPU 2165/3920  MEM 2296/6841
Не влезли: ['calico-node-brn8p', 'csi-node-driver-878fn', 'falcon-sensor-k5xzq', 'aws-node-termination-handler-g98fx', 'fluent-bit-kszcv', 'monitoring-prometheus-node-exporter-g9jch', 'compliance-benchmarker-x8s8c', 'fluentd-node-jpch7', 'Requests', '<none>']
